# 🧠 Introducción a LLM y Retrieval-Augmented Generation (RAG)


En este cuaderno, exploraremos el funcionamiento de los **Modelos de Lenguaje Grandes (LLM)** y cómo se integran con **Retrieval-Augmented Generation (RAG)** para enriquecer respuestas basadas en información externa.

**Objetivos**:
1. Entender cómo los LLM procesan texto.
2. Implementar un sistema básico de RAG.
3. Visualizar el flujo de datos desde la consulta hasta la respuesta.

---
**Requisitos previos**: 
- Familiaridad básica con Python.
- Conocimiento básico de NLP y redes neuronales.
---


In [11]:

# Instalar bibliotecas necesarias (descomentar si no están instaladas)
!pip install sentence-transformers transformers scikit-learn
!pip install shap


^C


## 🌀 Paso 1: Introducción a los LLM


Los Modelos de Lenguaje Grandes (LLM) son redes neuronales entrenadas en enormes cantidades de texto para predecir palabras o generar texto.  
Por ejemplo, dada una oración como *"El gato está en el"*, el modelo puede predecir *"tejado"* basándose en patrones aprendidos.

Los LLM utilizan:
- **Representaciones vectoriales**: Cada palabra/frase se convierte en un vector en un espacio multidimensional.
- **Mecanismo de atención**: Ayuda al modelo a centrarse en partes relevantes del texto.

A continuación, exploraremos cómo integrar un LLM con bases de datos para mejorar las respuestas usando RAG.


## 📂 Paso 2: Crear una base de datos ficticia

In [16]:

# Base de datos simulada
documents = [
    "La cúrcuma tiene propiedades antiinflamatorias.",
    "Se ha demostrado que la cúrcuma ayuda a la digestión.",
    "La cúrcuma contiene curcumina, un antioxidante."
]

# Mostrar los documentos
for i, doc in enumerate(documents, 1):
    print(f"Documento {i}: {doc}")


Documento 1: La cúrcuma tiene propiedades antiinflamatorias.
Documento 2: Se ha demostrado que la cúrcuma ayuda a la digestión.
Documento 3: La cúrcuma contiene curcumina, un antioxidante.


## 🔢 Paso 3: Transformar documentos en vectores

In [18]:
import numpy as np
import shap
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Modelo para embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar vectores para los documentos
document_vectors = model.encode(documents)

print("Vectores generados para los documentos (dimensiones):", document_vectors.shape)


Vectores generados para los documentos (dimensiones): (3, 384)


## 📝 Paso 4: Procesar la consulta del usuario

In [37]:

# Consulta del usuario
query = "¿Cómo ayuda la cúrcuma a la salud?"

# Transformar la consulta en un vector
query_vector = model.encode(query)
print("Vector de la consulta generado:", query_vector[:5], "...")


Vector de la consulta generado: [ 0.03837643  0.01184166 -0.06359773  0.01685266 -0.04091966] ...


In [42]:
import transformers
import shap

# load a transformers pipeline model
model = transformers.pipeline('sentiment-analysis', return_all_scores=True)

# explain the model on two sample inputs
explainer = shap.Explainer(model)
shap_values = explainer(["What a great movie! ...if you have no taste."])

# visualize the first prediction's explanation for the POSITIVE output class
shap.plots.text(shap_values[0, :, "POSITIVE"])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

d:\Programs\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\frany\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

d:\Programs\Anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


  0%|          | 0/240 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:22, 22.99s/it]               


## 🔍 Paso 5: Buscar documentos relevantes

In [ ]:

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calcular similitud coseno
similarities = cosine_similarity([query_vector], document_vectors)[0]

# Recuperar los documentos más similares
top_indices = np.argsort(similarities)[::-1][:2]  # Los 2 más relevantes
relevant_docs = [documents[i] for i in top_indices]

# Mostrar los documentos más relevantes
print("Documentos relevantes:")
for doc in relevant_docs:
    print("-", doc)


Documentos relevantes:
- Se ha demostrado que la cúrcuma ayuda a la digestión.
- La cúrcuma tiene propiedades antiinflamatorias.


## 💡 Paso 6: Generar respuesta con el LLM

In [ ]:

from transformers import pipeline

# Cargar el modelo LLM
qa_pipeline = pipeline("text2text-generation", model="t5-small")

# Concatenar los documentos relevantes
context = " ".join(relevant_docs)
final_input = f"Contexto: {context} Pregunta: {query}"

# Generar respuesta
response = qa_pipeline(final_input)
print("Respuesta generada:")
print(response[0]['generated_text'])


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

d:\Programs\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\frany\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

d:\Programs\Anaconda3\Lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Respuesta generada:
ayuda la crcuma a la digesti



## 🎯 Conclusión
En este cuaderno, hemos explorado cómo combinar **LLM** con una base de datos mediante **RAG**. Este enfoque permite enriquecer respuestas aprovechando tanto el conocimiento general del modelo como información específica de una base de datos. 🚀
